# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 3h Sleeping Cerebral Cortex Expression values and Group 2 be 3h SD Cerebral Cortex Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_CerCx_3h_'   # define a prefix to add to column names (making indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE6514.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 137)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 137)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'adjp-HypoT_12hS_vs_HypoT_12hSD', 'GSM149636_HypoT_12hSD.CEL',
       'GSM149637_HypoT_12hSD.CEL', 'GSM149648_HypoT_12hSD.CEL',
       'GSM149649_HypoT_12hSD.CEL', 'GSM149650_HypoT_12hSD.CEL',
       'avg-HypoT_12hSD', 'ANOVA-rawp', 'ANOVA-adjp', 'largest fold'],
      dtype='object', length=137)

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='CerCx_3hS.CEL'
sd_filt ='CerCx_3hSD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM149516_CerCx_3hS.CEL,GSM149517_CerCx_3hS.CEL,GSM149518_CerCx_3hS.CEL,GSM149519_CerCx_3hS.CEL,GSM149520_CerCx_3hS.CEL
Probesets,,,,,
1427138_at,6.83779,6.95928,7.06002,6.96113,7.25906
1425600_a_at,7.74728,7.86264,8.33633,8.24763,8.02129
1457168_at,5.56287,5.70365,5.69726,5.68221,5.83961
1450135_at,5.68220,5.69950,5.58693,5.64612,5.72589
1424014_at,8.27595,8.14901,8.25678,8.38142,7.90874


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM149521_CerCx_3hSD.CEL,GSM149522_CerCx_3hSD.CEL,GSM149523_CerCx_3hSD.CEL,GSM149524_CerCx_3hSD.CEL,GSM149525_CerCx_3hSD.CEL
Probesets,,,,,
1427138_at,7.08750,7.02883,6.83417,6.82263,7.09523
1425600_a_at,8.39086,8.48583,8.16422,7.31817,7.52341
1457168_at,5.77239,5.70869,5.80491,5.38684,5.56720
1450135_at,5.74461,5.51991,5.50637,5.35681,5.58668
1424014_at,8.43876,8.35439,8.53398,8.34201,8.01176


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.041784
1425600_a_at   -0.066536
1457168_at     -0.049114
1450135_at     -0.125252
1424014_at      0.141800
Name: 430AV2_CerCx_3h_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -0.284922
1425600_a_at   -0.161766
1457168_at     -0.350477
1450135_at     -1.176372
1424014_at      0.752012
Name: 430AV2_CerCx_3h_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.261179
1425600_a_at   -0.148286
1457168_at     -0.321271
1450135_at     -1.078341
1424014_at      0.689344
Name: 430AV2_CerCx_3h_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.404059
1425600_a_at    0.401308
1457168_at      0.406142
1450135_at      0.469193
1424014_at      0.428276
Name: 430AV2_CerCx_3h_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_CerCx_3h_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_CerCx_3h_Enrich,430AV2_CerCx_3h_poolStDev,430AV2_CerCx_3h_Cohens_d,430AV2_CerCx_3h_J,430AV2_CerCx_3h_Hedges_g,430AV2_CerCx_3h_Var_d,430AV2_CerCx_3h_Var_g,430AV2_CerCx_3h_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1436585_at,BB182297,expressed sequence BB182297,NaN,99463,NaN,NaN,NaN,NaN,NaN,NaN,...,8.706005e-08,0.901788,0.332394,0.037890,8.772508,0.916667,8.041466,4.247845,3.569370,1.889278
1417602_at,Per2,period homolog 2 (Drosophila),ENSMUSG00000055866,18627,NaN,circadian rhythm // transcription // signal tr...,two-component sensor activity // protein bindi...,cytoplasm // nucleus,GenMAPP-Circadian_Exercise // Diurnally Regula...,"mmu-miR-1(miRanda), mmu-miR-103(miRanda), mmu-...",...,4.528292e-19,1.299228,0.899836,0.126790,7.097061,0.916667,6.505639,2.918414,2.452278,1.565975
1429169_at,Rbm3,RNA binding motif protein 3 [Source:MGI Symbol...,ENSMUSG00000031167,19652,NaN,translation // response to cold // gene silenc...,RNA binding // nucleic acid binding // nucleot...,NaN,GenMAPP-mRNA_processing_binding_Reactome // mR...,"mmu-miR-1(miRanda), mmu-miR-1192(miRanda), mmu...",...,1.205980e-15,1.316408,1.216108,0.192921,6.303647,0.916667,5.778343,2.386798,2.005574,1.416183
1446090_at,NaN,NaN,NaN,NaN,Mm.393511,NaN,NaN,NaN,NaN,NaN,...,5.221521e-07,0.587210,0.440070,0.075504,5.828431,0.916667,5.342729,2.098531,1.763349,1.327911
1455166_at,Arl5b,ADP-ribosylation factor-like 5B [Source:MGI Sy...,ENSMUSG00000017418,75869,NaN,small GTPase mediated signal transduction,nucleotide binding // GTP binding,intracellular,NaN,"mmu-let-7e(miRanda), mmu-let-7f(miRanda), mmu-...",...,9.603164e-09,0.916788,0.433884,0.075233,5.767205,0.916667,5.286604,2.063033,1.733520,1.316632
1421962_at,Dnajb5,"DnaJ (Hsp40) homolog, subfamily B, member 5",ENSMUSG00000036052,56323,NaN,protein folding,heat shock protein binding // unfolded protein...,NaN,NaN,"mmu-let-7a(miRanda), mmu-let-7b(RNAhybrid|miRa...",...,5.236422e-27,1.479322,0.634068,0.110231,5.752199,0.916667,5.272849,2.054390,1.726258,1.313871
1446598_at,NaN,NaN,NaN,NaN,Mm.441616,NaN,NaN,NaN,NaN,NaN,...,4.863846e-16,0.953088,0.278166,0.050634,5.493707,0.916667,5.035898,1.909041,1.604124,1.266540
1428052_a_at,Zmym1,"zinc finger, MYM domain containing 1",ENSMUSG00000043872,68310,NaN,NaN,protein dimerization activity // zinc ion binding,nucleus,NaN,"mmu-miR-103(miRanda), mmu-miR-107(miRanda), mm...",...,1.020090e-11,1.599828,0.813876,0.157443,5.169325,0.916667,4.738548,1.736096,1.458803,1.207809
1420500_at,Dnajc1,"DnaJ (Hsp40) homolog, subfamily C, member 1",ENSMUSG00000026740,13418,NaN,regulation of protein secretion // protein fol...,heat shock protein binding // protein binding ...,endoplasmic reticulum membrane // microsome //...,NaN,"mmu-let-7(TargetScan), mmu-let-7a(miRanda|pict...",...,8.670196e-07,0.679102,0.454128,0.088709,5.119316,0.916667,4.692706,1.710370,1.437186,1.198827


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites',
       ...
       'ANOVA-adjp', 'largest fold', '430AV2_CerCx_3h_Enrich',
       '430AV2_CerCx_3h_poolStDev', '430AV2_CerCx_3h_Cohens_d',
       '430AV2_CerCx_3h_J', '430AV2_CerCx_3h_Hedges_g',
       '430AV2_CerCx_3h_Var_d', '430AV2_CerCx_3h_Var_g',
       '430AV2_CerCx_3h_SEg'],
      dtype='object', length=145)

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM149516_CerCx_3hS.CEL,GSM149517_CerCx_3hS.CEL,GSM149518_CerCx_3hS.CEL,GSM149519_CerCx_3hS.CEL,GSM149520_CerCx_3hS.CEL,avg-CerCx_3hS,log_fold-CerCx_3hS_vs_CerCx_3hSD,fold-CerCx_3hS_vs_CerCx_3hSD,rawp-CerCx_3hS_vs_CerCx_3hSD,adjp-CerCx_3hS_vs_CerCx_3hSD,...,ANOVA-adjp,largest fold,430AV2_CerCx_3h_Enrich,430AV2_CerCx_3h_poolStDev,430AV2_CerCx_3h_Cohens_d,430AV2_CerCx_3h_J,430AV2_CerCx_3h_Hedges_g,430AV2_CerCx_3h_Var_d,430AV2_CerCx_3h_Var_g,430AV2_CerCx_3h_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_CerCx_3h_Enrich',u'430AV2_CerCx_3h_Hedges_g', u'430AV2_CerCx_3h_Var_g', u'430AV2_CerCx_3h_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_CerCx_3h_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_CerCx_3h_Enrich,430AV2_CerCx_3h_Hedges_g,430AV2_CerCx_3h_Var_g,430AV2_CerCx_3h_SEg
MGI symbol,,,,
0610005C13Rik,-0.030064,-0.295197,0.340468,0.583497
0610008F07Rik,0.009456,0.073185,0.336379,0.579982
0610009B22Rik,-0.004386,-0.024466,0.336141,0.579777
0610009D07Rik,0.001255,-0.000852,0.336167,0.579799
0610009O20Rik,-0.126194,-1.290894,0.419431,0.647635
0610010K14Rik,0.068716,0.287317,0.340239,0.583300
0610012G03Rik,-0.121502,-0.809297,0.400266,0.629804
0610031J06Rik,-0.148844,-1.248059,0.413994,0.643423
0610037L13Rik,-0.027654,-0.300703,0.340632,0.583637
